In [17]:
from PIL import Image
import pickle
import numpy as np
import os
from core.generator import Generator
from skimage import transform
from matplotlib import pyplot as plt
from matplotlib import cm
import itertools

In [2]:
fcorrs = 'corrs.pkl'
ffeats = 'test_feats_{}.pkl'

In [75]:
input_shape = [256, 256, 1]
n_classes = 2
batch_size = 32
images_are_colored = input_shape[-1] == 3
train_file = 'train.txt'
test_file = 'test.txt'
model_name = 'inceptionv3'

absp = '../__ct_lungs'

In [76]:
test_generator = Generator(os.path.join(absp, test_file), batch_size=batch_size, colored=images_are_colored)

In [77]:
with open(os.path.join(absp, model_name, 'corrs.pkl'), 'rb') as file:
    corrs = pickle.load(file)
corrs = np.array(corrs)
corrs[np.isnan(corrs)] = 0
corrs = np.sign(corrs) * np.square(corrs)

In [78]:
s = 0
with open(os.path.join(absp, model_name, 'feats', 'test_feats_{}.pkl'.format(s)), 'rb') as file:
    test_feats = pickle.load(file)

step = len(test_feats)
X, y = test_generator[s * step + 1]
X_feats = test_feats[0: len(X)]

In [79]:
def imresize(img, new_shape):
    dtype = img.dtype
    mult = np.abs(img).max() * 2
    m = img.astype(np.float32) / mult
    m = transform.resize(m, new_shape, order=3, mode='constant')
    m = m * mult
    return m.astype(dtype)

In [80]:
def get_image(img):
    im = (img * 255.0).astype(np.uint8)
    if img.shape[-1] == 1 or len(img.shape) == 2:
        return Image.fromarray(np.squeeze(im), mode='L')
    else:
        return Image.fromarray(im, mode='RGB')
    
def show_images(images, titles = None, cols = 1):
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(np.ceil(n_images/float(cols)), cols, n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title, size=80)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [81]:
heat_maps = np.matmul(X_feats, corrs)
heat_maps = np.array([imresize(hm, X[0].shape[:2]) for hm in heat_maps])

heat_maps = [(hm - hm.min()) / (hm.max() - hm.min()) for hm in heat_maps]
heat_maps = [cm.jet(hm) for hm in heat_maps]

startt = 0
endd = 32
imgs = list(itertools.chain(*zip(np.squeeze(X), heat_maps)))
show_images(imgs[startt: endd], cols=4)

In [82]:
def save_img(x, fname):
    if len(x.shape) > 2:
        img = Image.fromarray((x * 255.0).astype(np.uint8))
    else:
        img = Image.fromarray((x * 255.0).astype(np.uint8), mode='L')
    img.save(fname)

In [83]:
fname = os.path.join('plots', 'hms', 'ct_{}.png')
hmfname = os.path.join('plots', 'hms', 'ct_{}_hm.png')
for i, x in enumerate(np.squeeze(X)):
    save_img(x, fname.format(i + 1))
    
for i, x in enumerate(heat_maps):
    save_img(x, hmfname.format(i + 1))